# Library

In [2]:
import pandas as pd
from pmaw import PushshiftAPI
api = PushshiftAPI()
import datetime as dt
import numpy as np
import praw
import time
#from redditbots_function import flatten_author, author_comments

# Set up

In [3]:
# Praw Enhancement
reddit = praw.Reddit(
 client_id='9wXh5oa4cfW07_eUn5Hu3A',
 client_secret='m3GnhaEvbM5LGCWX3BMghLCatOLN3g',
 user_agent=f'python: PMAW request enrichment (by u/softyarn)'
)
api_praw = PushshiftAPI(praw=reddit)
# Pmaw
api = PushshiftAPI(max_sleep = 60, shards_down_behavior = None)

# Function

In [4]:
def get_epochdate(bot_epoch, duration):
    before = int((dt.datetime.fromtimestamp(bot_epoch)+dt.timedelta(days = duration)).timestamp())
    after = int((dt.datetime.fromtimestamp(bot_epoch)-dt.timedelta(days = duration)).timestamp())
    return before, after
def within60days(df, bot_epoch, duration):
    before, after = get_epochdate(bot_epoch = bot_epoch, duration = duration)
    res = df.loc[(df['created_utc'] >= after) & (df['created_utc'] <= before)]
    return res
def author_list(df):
    nomod = df.loc[(df['author'] != "AutoModerator") & (df['author'] != '[deleted]')]
    authors = nomod.loc[:,'author']
    authors_unique = np.unique(authors)
    return authors_unique
def author_comments(df, bot_epoch, duration, limit = 10000000000):
    before, after = get_epochdate(bot_epoch, duration) 
    #creating loop so that only 100 authors per praw
    tic = time.time()
    global res
    res = []
    authors = author_list(df)
    i = 0
    while i < len(authors):
        j = min(i+2, len(authors))
        x = ','.join(authors[i:j])
        comments = api.search_comments(author = x, limit = limit, before = before, after = after, filter = ['id', 'banned_at_utc', 'mod_reason_title', 'author', 'created_utc', 'parent_id', 'subreddit_id', 'body'], mem_safe= True)
        res += [c for c in comments]
        if i%500 == 0:
            res_out = pd.DataFrame(res)
            res_out.to_csv('author_comments_' + str(i/1000) + '.csv')
            res = []
        i +=2
        if i%100 == 0:
            time.sleep(1)
            print('Time elapsed: ', round(time.time() - tic),' secs, i = ',i)
    res_out = pd.DataFrame(res)
    res_out.to_csv('author_comments_Finale.csv')   
    return res_out

# Running

In [5]:
fds = pd.read_csv("E:/gihub-data/redditbots/fds/fds_comments.csv")

C:\Users\nguye\AppData\Local\Temp\ipykernel_4964\3957158027.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  fds = pd.read_csv("E:/gihub-data/redditbots/fds/fds_comments.csv")


In [6]:
fds1 = within60days(fds, bot_epoch = 1572234835 , duration = 30)

In [7]:
authors = author_list(fds1)

In [8]:
df_res = author_comments(df = fds1, bot_epoch = 1572234835, duration = 30, limit = None)

Time elapsed:  663  secs, i =  100
Time elapsed:  1281  secs, i =  200
Time elapsed:  1919  secs, i =  300
Time elapsed:  2758  secs, i =  400
Time elapsed:  3655  secs, i =  500
Time elapsed:  4260  secs, i =  600
Time elapsed:  4885  secs, i =  700
Time elapsed:  5499  secs, i =  800
Time elapsed:  6150  secs, i =  900
Time elapsed:  6732  secs, i =  1000
Time elapsed:  7325  secs, i =  1100
Time elapsed:  8140  secs, i =  1200
Time elapsed:  8735  secs, i =  1300
Time elapsed:  9373  secs, i =  1400
Time elapsed:  9966  secs, i =  1500
Time elapsed:  10635  secs, i =  1600
Time elapsed:  11204  secs, i =  1700
Time elapsed:  12252  secs, i =  1800
Time elapsed:  12865  secs, i =  1900
Time elapsed:  13505  secs, i =  2000
Time elapsed:  14114  secs, i =  2100
Time elapsed:  15372  secs, i =  2200
Time elapsed:  16014  secs, i =  2300
Time elapsed:  16923  secs, i =  2400
Time elapsed:  17683  secs, i =  2500
Time elapsed:  18339  secs, i =  2600
Time elapsed:  19201  secs, i =  2700

ConnectionError: HTTPSConnectionPool(host='api.pushshift.io', port=443): Max retries exceeded with url: /reddit/comment/search?author=tequilathehun%2Ctest1729&before=1569805256&after=1569800949&filter=id&filter=banned_at_utc&filter=mod_reason_title&filter=author&filter=created_utc&filter=parent_id&filter=subreddit_id&filter=body&size=100&sort=desc&metadata=true (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002151B8852B0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

# Testing 

In [3]:
open("E:/github/redditbots/author_comments_test.csv", encoding = 'utf-8-sig')

<_io.TextIOWrapper name='E:/github/redditbots/author_comments_test.csv' mode='r' encoding='utf-8-sig'>

In [6]:
before, after = get_epochdate(bot_epoch = 1572234835, duration = 30)
i = 0 
res = []
authors = author_list(fds1)
limit = 10000000000

In [7]:
tic = time.time()
while i < len(authors):
        j = min(i+2, len(authors))
        x = ','.join(authors[i:j])
        comments = api.search_comments(author = x, limit = limit, before = before, after = after, filter = ['id', 'banned_at_utc', 'mod_reason_title', 'author', 'created_utc', 'parent_id', 'subreddit_id', 'body'], mem_safe= True)
        res += [c for c in comments]
        i +=2
        if i%5 == 0:
            time.sleep(1)
            print('Time elapsed: ', round(time.time() - tic),' secs, i = ',i)

Time elapsed:  53  secs, i =  10
Time elapsed:  116  secs, i =  20
Time elapsed:  176  secs, i =  30


In [8]:
res_df = pd.DataFrame(res)

In [7]:
comments1 = api.search_comments(author = 'AverageToHot,dwxek', limit = limit, before = before, after = after, filter = ['id', 'banned_at_utc', 'mod_reason_title', 'author', 'created_utc', 'parent_id', 'subreddit_id', 'body'], mem_safe= True)
comments1_list = [c for c in comments1]

In [8]:
comments2 = api.search_comments(author = 'extraacct1234,randomgirlimok', limit = limit, before = before, after = after, filter = ['id', 'banned_at_utc', 'mod_reason_title', 'author', 'created_utc', 'parent_id', 'subreddit_id', 'body'], mem_safe= True)
comments2_list = [c for c in comments2]

In [14]:
comments1_df = pd.DataFrame(comments1_list)
comments2_df = pd.DataFrame(comments2_list)